In [27]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
path_svm =  'Models/best_knnc.pickle'
with open(path_svm, 'rb') as data:
    svc_model = pickle.load(data)

path_tfidf = "Pickles/tfidf.pickle"
with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

path_df = "Pickles/df.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)
    

path_category_names = "Pickles/category_names.pickle"
with open(path_category_names, 'rb') as data:
    category_names = pickle.load(data)
    
path_category_codes = "Pickles/category_codes.pickle"
with open(path_category_codes, 'rb') as data:
    category_codes = pickle.load(data)

In [29]:
punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))

def create_features_from_text(text):
    
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['Content'])
    df.loc[0] = text
    df['Content_Clean'] = df['Content'].str.replace("\r", " ")
    df['Content_Clean'] = df['Content_Clean'].str.replace("\n", " ")
    df['Content_Clean'] = df['Content_Clean'].str.replace("    ", " ")
    df['Content_Clean'] = df['Content_Clean'].str.replace('"', '')
    df['Content_Clean'] = df['Content_Clean'].str.lower()
    df['Content_Clean'] = df['Content_Clean']
    for punct_sign in punctuation_signs:
        df['Content_Clean'] = df['Content_Clean'].str.replace(punct_sign, '')
    df['Content_Clean'] = df['Content_Clean'].str.replace("'s", "")
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    text = df.loc[0]['Content_Clean']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)    
    lemmatized_text_list.append(lemmatized_text)
    df['Content_Clean'] = lemmatized_text_list
    df['Content_Clean'] = df['Content_Clean']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Clean'] = df['Content_Clean'].str.replace(regex_stopword, '')
    df = df['Content_Clean']
    #df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
    #df = df.rename(columns={"A": "a", "B": "c"})
    # TF-IDF
    features = tfidf.transform(df).toarray()
    
    return features

In [30]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [49]:
def predict_from_text(text,debug):
    
    # Predict using the input model
    prediction_svc = svc_model.predict(create_features_from_text(text))[0]
    prediction_svc_proba = svc_model.predict_proba(create_features_from_text(text))[0]
    
    # Return result
    category_svc = get_category_name(prediction_svc)
    scoring = prediction_svc_proba.max()*100
    if debug:
        print("The predicted category using the KNN model is %s." %(category_svc) )
        print("The conditional probability is: %a" %(scoring))
    
    return prediction_svc,scoring,category_svc

In [50]:
predict_from_text("yushi angle transducer ",True)

The predicted category using the KNN model is None.
The conditional probability is: 50.0


(4, 50.0, 'None')

In [51]:
TODO =df[df.group == "None"]
TODO["Hypothese"] = TODO["Title_Clean"].apply(lambda x: predict_from_text(str(x),False))
TODO[['new_group','confidence','new_category']] = pd.DataFrame(TODO.Hypothese.values.tolist(), index= TODO.index)
TODO

/home/kelu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/kelu/.local/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ItemID,value,_currencyID,ViewItemURLForNaturalSearch,Title,PrimaryCategoryID,Country,Description,group,StrPrimaryCategoryID,Content,Title_Clean,Category_Code,Hypothese,new_group,confidence,new_category
12,254459167401,1999.00,USD,https://www.ebay.com/itm/Olympus-Sonic-1200S-U...,category181989 Olympus Sonic 1200S Ultrasonic ...,181989,US,Ultrasonic Flaw DetectorStock #191108NIncluded...,None,category181989,category181989 Olympus Sonic 1200S Ultrasonic ...,category181989 olympus sonic 1200s ultrasonic ...,4,"(6, 75.0, NDT)",6,75.0,NDT
45,223489256740,510.56,USD,https://www.ebay.com/itm/Siemens-VF13-5SP-Line...,category184542 Siemens VF13-5SP Linear Array ...,184542,GB,Frequency:13-5MHz Compatible system: Antares/X...,None,category184542,category184542 Siemens VF13-5SP Linear Array ...,category184542 siemens vf135sp linear array u...,4,"(1, 75.0, Array)",1,75.0,Array
59,264280176588,220.00,USD,https://www.ebay.com/itm/SALE-Ultrasound-trans...,category181989 SALE! Ultrasound transducer Kra...,181989,RU,"New, unexpended, working.Type: DA503;Contact D...",None,category181989,category181989 SALE! Ultrasound transducer Kra...,category181989 sale ultrasound transducer krau...,4,"(4, 50.0, None)",4,50.0,None
68,184188801780,175.75,USD,https://www.ebay.com/itm/CTG-HP-8031-A-Toko-u-...,"category30827 CTG HP 8031 A, Toko- u. Ultrasch...",30827,DE,"CTG HP 8031 A, 2 Sonden (Toko- + Ultraschalls...",None,category30827,"category30827 CTG HP 8031 A, Toko- u. Ultrasch...",category30827 ctg hp 8031 toko u. ultraschall...,4,"(1, 75.0, Array)",1,75.0,Array
218,352992739775,239.00,USD,https://www.ebay.com/itm/BK-Medical-Ultrasound...,category184515 BK Medical Ultrasound PROBE Tra...,184515,US,B-K Medical Ultrasound PROBE Transducer IP 57 ...,None,category184515,category184515 BK Medical Ultrasound PROBE Tra...,category184515 bk medical ultrasound probe tra...,4,"(1, 75.0, Array)",1,75.0,Array
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3866,352992713134,1800.00,USD,https://www.ebay.com/itm/Medison-3D4-8ET-Ultra...,category184515 Medison 3D4-8ET Ultrasound Prob...,184515,US,Medison 3D4-8ET Ultrasound Probe / Transducer ...,None,category184515,category184515 Medison 3D4-8ET Ultrasound Prob...,category184515 medison 3d48et ultrasound probe...,4,"(1, 100.0, Array)",1,100.0,Array
3951,223935245621,3595.43,USD,https://www.ebay.com/itm/GE-RAB2-5-D-Ultrasoun...,category184515 GE RAB2-5-D Ultrasound Probe / ...,184515,US,"GE RAB2-5-D ""The sale of this item may be sub...",None,category184515,category184515 GE RAB2-5-D Ultrasound Probe / ...,category184515 ge rab25d ultrasound probe tra...,4,"(1, 100.0, Array)",1,100.0,Array
3976,264651216282,2180.00,USD,https://www.ebay.com/itm/Hitachi-EUP-S50-EUP-S...,category3149 Hitachi EUP-S50 / EUP-S50S ultra...,3149,CN,30 days warrantyDHL courier shippingWe also of...,None,category3149,category3149 Hitachi EUP-S50 / EUP-S50S ultra...,category3149 hitachi eups50 eups50s ultrasou...,4,"(1, 75.0, Array)",1,75.0,Array
3982,193302119564,2651.82,USD,https://www.ebay.com/itm/Storz-276100-20-Calcu...,category185170 Storz 276100-20 Calcuson à Ultr...,185170,US,Les commandes réalisées à partir des États-Uni...,None,category185170,category185170 Storz 276100-20 Calcuson à Ultr...,category185170 storz 27610020 calcuson à ultra...,4,"(2, 75.0, NotUS)",2,75.0,NotUS
